In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Input
from tensorflow.keras.models import Model

In [3]:
SAMPLE_SIZE = 10000

In [4]:
with open('data/board_states.npy', 'rb') as boards_file, \
    open('data/winners.npy', 'rb') as winners_file:
    boards = np.load(boards_file)
    shape = boards.shape
    boards = np.array([np.subtract(x[:,:,0], x[:,:,1]) for x in boards]).reshape((shape[0], shape[1], shape[2], 1))
    winners = np.load(winners_file)

#     winners_cat = np.array([[1.0, 0.0] if x==1 else [0.0, 1.0] for x in winners])

#     print(boards[:SAMPLE_SIZE, :, :, :].shape, winners_cat[:SAMPLE_SIZE, :].shape)
    print(boards[:SAMPLE_SIZE, :, :, :].shape, winners[:SAMPLE_SIZE, :].shape)
    
#     nums = np.arange(len(winners_cat))

    nums = np.arange(len(winners))
    np.random.shuffle(nums)
    X_train =  boards[nums[:SAMPLE_SIZE],:,:,:]
#     y_train = winners_cat[nums[:SAMPLE_SIZE],:]
    y_train = winners[nums[:SAMPLE_SIZE],:]
    
    X_test =  boards[nums[SAMPLE_SIZE:(2*SAMPLE_SIZE)],:,:,:]
#     y_test = winners_cat[nums[SAMPLE_SIZE:(2*SAMPLE_SIZE)],:]
    y_test = winners[nums[SAMPLE_SIZE:(2*SAMPLE_SIZE)],:]

(10000, 13, 13, 1) (10000, 1)


In [5]:

y_train[0:30]

array([[0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0]])

### Test loss =

In [6]:
learning_rates = np.exp(np.arange(-10, -2, 1))
learning_rates

array([  4.53999298e-05,   1.23409804e-04,   3.35462628e-04,
         9.11881966e-04,   2.47875218e-03,   6.73794700e-03,
         1.83156389e-02,   4.97870684e-02])

In [7]:
epochs_list = np.arange(1,11)
epochs_list

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [8]:
BATCH_SIZE = 1000
NUM_EPOCHS = 10
#STEPS_PER_EPOCH = 30
START_LEARNING_RATE = 0.0025

def model_function(learning_rate = START_LEARNING_RATE):
    # a layer instance is callable on a tensor, and returns a tensor
    model = tf.keras.Sequential([
        Conv2D(100, kernel_size=(2, 2), strides=(1, 1),
                      data_format='channels_last',
                      activation='relu',
                      input_shape=(13, 13, 1)),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Flatten(),
        Dense(50, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate),
              loss=tf.keras.losses.binary_crossentropy,       # mean squared error
              metrics=['accuracy'])  # mean absolute error
    
    return model

In [ ]:
losses = []
for learning_rate in learning_rates:
    for num_epochs in epochs_list:
        BATCH_SIZE = 1000
#         NUM_EPOCHS = 10
        #STEPS_PER_EPOCH = 30
        model = model_function(learning_rate)

        model.fit(X_train,y_train, verbose=1, batch_size = BATCH_SIZE,
                  epochs=num_epochs)
        evaluate = model.evaluate(X_test, y_test, batch_size = BATCH_SIZE)
        losses.append((learning_rate, num_epochs, evaluate[0]))

Epoch 1/1
10000/10000 [==============================] - 1s 150us/step
Epoch 1/2
10000/10000 [==============================] - 4s 358us/step - loss: 0.6918 - acc: 0.5137
Epoch 2/2
10000/10000 [==============================] - 2s 181us/step
Epoch 1/3
10000/10000 [==============================] - 4s 396us/step - loss: 0.6902 - acc: 0.5329
Epoch 2/3
10000/10000 [==============================] - 3s 298us/step - loss: 0.6858 - acc: 0.5634
Epoch 3/3
10000/10000 [==============================] - 2s 153us/step
Epoch 1/4
10000/10000 [==============================] - 4s 375us/step - loss: 0.6934 - acc: 0.4997
Epoch 2/4
10000/10000 [==============================] - 3s 260us/step - loss: 0.6897 - acc: 0.5379
Epoch 3/4
10000/10000 [==============================] - 3s 307us/step - loss: 0.6863 - acc: 0.5610
Epoch 4/4
10000/10000 [==============================] - 2s 165us/step
Epoch 1/5
10000/10000 [==============================] - 4s 385us/step - loss: 0.6916 - acc: 0.5212
Epoch 2/5
10000/

10000/10000 [==============================] - 3s 265us/step - loss: 0.6746 - acc: 0.5796
Epoch 4/6
10000/10000 [==============================] - 3s 271us/step - loss: 0.6693 - acc: 0.5840
Epoch 5/6
10000/10000 [==============================] - 3s 265us/step - loss: 0.6650 - acc: 0.5914
Epoch 6/6
10000/10000 [==============================] - 2s 185us/step
Epoch 1/7
10000/10000 [==============================] - 5s 495us/step - loss: 0.6901 - acc: 0.5313
Epoch 2/7
10000/10000 [==============================] - 3s 295us/step - loss: 0.6804 - acc: 0.5724
Epoch 3/7
10000/10000 [==============================] - 3s 280us/step - loss: 0.6723 - acc: 0.5880
Epoch 4/7
10000/10000 [==============================] - 3s 320us/step - loss: 0.6665 - acc: 0.5927
Epoch 5/7
10000/10000 [==============================] - 3s 273us/step - loss: 0.6622 - acc: 0.5985
Epoch 6/7
10000/10000 [==============================] - 3s 255us/step - loss: 0.6581 - acc: 0.6056
Epoch 7/7
 7000/10000 [================

Starting with the model above:
ephochs = 10
learning_rate = 0.0024787521766663585
loss = 0.22040595710277558

In [ ]:
min(losses, key = lambda t: t[2])

### Tying 1, 2, or 3 Channels and Convolution Size

In [10]:
with open('data/board_states.npy', 'rb') as boards_file, \
    open('data/winners.npy', 'rb') as winners_file:
    boards = np.load(boards_file)
    shape = boards.shape
    boards = np.array([np.subtract(x[:,:,0], x[:,:,1]) for x in boards]).reshape((shape[0], shape[1], shape[2], 1))
    
    #2 or 3 channels
    boards_2 = np.tile(boards,2)
    boards_3 = np.tile(boards,3)
    
#     winners = np.load(winners_file)
#     print(boards[:SAMPLE_SIZE, :, :, :].shape, winners[:SAMPLE_SIZE, :].shape)
    
    X_train_2 = boards_2[nums[:SAMPLE_SIZE],:,:,:]
    X_train_3 = boards_3[nums[:SAMPLE_SIZE],:,:,:]

#     y_train = winners[nums[:SAMPLE_SIZE],:]
    
    X_test_2 = boards_2[nums[SAMPLE_SIZE:(2*SAMPLE_SIZE)],:,:,:]
    X_test_3 = boards_3[nums[SAMPLE_SIZE:(2*SAMPLE_SIZE)],:,:,:]
    
#     y_test = winners[nums[SAMPLE_SIZE:(2*SAMPLE_SIZE)],:]

In [11]:
START_LEARNING_RATE = 0.0024787521766663585
NUM_EPOCHS = 10
BATCH_SIZE = 1000

def model_function(conv=100, channels = 1):
    # a layer instance is callable on a tensor, and returns a tensor
    model = tf.keras.Sequential([
        Conv2D(conv, kernel_size=(2, 2), strides=(1, 1),
                      data_format='channels_last',
                      activation='relu',
                      input_shape=(13, 13, channels)),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Flatten(),
        Dense(50, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(START_LEARNING_RATE),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
    
    return model

In [12]:
conv2d_size = [int(x) for x in [2**y for y in np.arange(1, 9, 1)]]
losses_2 = []
X_trains = [X_train, X_train_2, X_train_3]
X_tests = [X_test, X_test_2, X_test_3]

In [16]:
# for i in range(3):
#     print("CHANNELS: ", i+1)
#     for conv in conv2d_size:
#         print("CONV: ", conv)
#         model = model_function(conv, i+1)

#         model.fit(X_trains[i],y_train, verbose=1, batch_size = BATCH_SIZE,
#                   epochs=NUM_EPOCHS)
#         evaluate = model.evaluate(X_tests[i], y_test, batch_size = BATCH_SIZE)
#         losses_2.append((i+1, conv, evaluate[0]))

CHANNELS:  1
CONV:  2
Epoch 1/10
10000/10000 [==============================] - 1s 68us/step - loss: 0.2512 - mean_absolute_error: 0.4991
Epoch 2/10
10000/10000 [==============================] - 0s 10us/step - loss: 0.2441 - mean_absolute_error: 0.4910
Epoch 3/10
10000/10000 [==============================] - 0s 9us/step - loss: 0.2409 - mean_absolute_error: 0.4836
Epoch 4/10
10000/10000 [==============================] - 0s 10us/step - loss: 0.2388 - mean_absolute_error: 0.4792
Epoch 5/10
10000/10000 [==============================] - 0s 9us/step - loss: 0.2367 - mean_absolute_error: 0.4765
Epoch 6/10
10000/10000 [==============================] - 0s 9us/step - loss: 0.2350 - mean_absolute_error: 0.4741
Epoch 7/10
10000/10000 [==============================] - 0s 9us/step - loss: 0.2331 - mean_absolute_error: 0.4711
Epoch 8/10
10000/10000 [==============================] - 0s 9us/step - loss: 0.2313 - mean_absolute_error: 0.4676
Epoch 9/10
10000/10000 [==============================]

10000/10000 [==============================] - 1s 62us/step - loss: 0.2293 - mean_absolute_error: 0.4613
Epoch 4/10
10000/10000 [==============================] - 1s 75us/step - loss: 0.2238 - mean_absolute_error: 0.4515
Epoch 5/10
10000/10000 [==============================] - 1s 75us/step - loss: 0.2206 - mean_absolute_error: 0.4448
Epoch 6/10
10000/10000 [==============================] - 1s 70us/step - loss: 0.2174 - mean_absolute_error: 0.4376
Epoch 7/10
10000/10000 [==============================] - 1s 61us/step - loss: 0.2148 - mean_absolute_error: 0.4362
Epoch 8/10
10000/10000 [==============================] - 1s 62us/step - loss: 0.2113 - mean_absolute_error: 0.4280
Epoch 9/10
10000/10000 [==============================] - 1s 61us/step - loss: 0.2097 - mean_absolute_error: 0.4272
Epoch 10/10
10000/10000 [==============================] - 1s 55us/step
CONV:  128
Epoch 1/10
10000/10000 [==============================] - 2s 215us/step - loss: 0.2524 - mean_absolute_error: 0.4932

In [17]:
# min(losses_2, key = lambda t: t[2])

(3, 16, 0.22155779749155044)

Channels = 3,  Conv Size = 16,  Loss= 0.22155779749155044

### Strides and Dense Size

In [13]:
START_LEARNING_RATE = 0.0024787521766663585
NUM_EPOCHS = 10
BATCH_SIZE = 1000

def model_function(dense, stride):
    # a layer instance is callable on a tensor, and returns a tensor
    model = tf.keras.Sequential([
        Conv2D(16, kernel_size=(2, 2), strides=stride,
                      data_format='channels_last',
                      activation='relu',
                      input_shape=(13, 13, 3)),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Flatten(),
        Dense(dense, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(START_LEARNING_RATE),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
    
    return model

In [14]:
dense_size = [int(x) for x in [2**y for y in np.arange(1, 9, 1)]]
strides = [(1,1), (1,2), (2,1), (2,2)]
losses_3 = []

In [23]:
# for dense in dense_size:
#     print("DENSE: ", dense)
#     for stride in strides:
#         print("STRIDE: ", stride)
#         model = model_function(dense, stride)

#         model.fit(X_train_3,y_train, verbose=1, batch_size = BATCH_SIZE,
#                   epochs=NUM_EPOCHS)
#         evaluate = model.evaluate(X_test_3, y_test, batch_size = BATCH_SIZE)
#         losses_3.append((dense, stride, evaluate[0]))

DENSE:  2
STRIDE:  (1, 1)
Epoch 1/10
10000/10000 [==============================] - 2s 164us/step - loss: 0.2467 - mean_absolute_error: 0.4941
Epoch 2/10
10000/10000 [==============================] - 0s 36us/step - loss: 0.2401 - mean_absolute_error: 0.4807
Epoch 3/10
10000/10000 [==============================] - 0s 33us/step - loss: 0.2358 - mean_absolute_error: 0.4730
Epoch 4/10
10000/10000 [==============================] - 0s 27us/step - loss: 0.2329 - mean_absolute_error: 0.4689
Epoch 5/10
10000/10000 [==============================] - 0s 19us/step - loss: 0.2306 - mean_absolute_error: 0.4651
Epoch 6/10
10000/10000 [==============================] - 0s 18us/step - loss: 0.2288 - mean_absolute_error: 0.4614
Epoch 7/10
10000/10000 [==============================] - 0s 20us/step - loss: 0.2275 - mean_absolute_error: 0.4571
Epoch 8/10
10000/10000 [==============================] - 0s 17us/step - loss: 0.2262 - mean_absolute_error: 0.4554
Epoch 9/10
10000/10000 [=====================

10000/10000 [==============================] - 0s 15us/step - loss: 0.2426 - mean_absolute_error: 0.4899
Epoch 3/10
10000/10000 [==============================] - 0s 15us/step - loss: 0.2372 - mean_absolute_error: 0.4786
Epoch 4/10
10000/10000 [==============================] - 0s 15us/step - loss: 0.2334 - mean_absolute_error: 0.4698
Epoch 5/10
10000/10000 [==============================] - 0s 16us/step - loss: 0.2305 - mean_absolute_error: 0.4638
Epoch 6/10
10000/10000 [==============================] - 0s 17us/step - loss: 0.2282 - mean_absolute_error: 0.4600
Epoch 7/10
10000/10000 [==============================] - 0s 16us/step - loss: 0.2265 - mean_absolute_error: 0.4566
Epoch 8/10
10000/10000 [==============================] - 0s 15us/step - loss: 0.2250 - mean_absolute_error: 0.4538
Epoch 9/10
10000/10000 [==============================] - 0s 17us/step - loss: 0.2233 - mean_absolute_error: 0.4510
Epoch 10/10
10000/10000 [==============================] - 1s 87us/step
STRIDE:  (2

10000/10000 [==============================] - 0s 18us/step - loss: 0.2337 - mean_absolute_error: 0.4707
Epoch 4/10
10000/10000 [==============================] - 0s 20us/step - loss: 0.2292 - mean_absolute_error: 0.4643
Epoch 5/10
10000/10000 [==============================] - 0s 19us/step - loss: 0.2256 - mean_absolute_error: 0.4573
Epoch 6/10
10000/10000 [==============================] - 0s 19us/step - loss: 0.2223 - mean_absolute_error: 0.4509
Epoch 7/10
10000/10000 [==============================] - 0s 18us/step - loss: 0.2179 - mean_absolute_error: 0.4442
Epoch 8/10
10000/10000 [==============================] - 0s 18us/step - loss: 0.2134 - mean_absolute_error: 0.4383
Epoch 9/10
10000/10000 [==============================] - 0s 18us/step - loss: 0.2085 - mean_absolute_error: 0.4320
Epoch 10/10
10000/10000 [==============================] - 1s 107us/step
STRIDE:  (2, 2)
Epoch 1/10
10000/10000 [==============================] - 2s 243us/step - loss: 0.2499 - mean_absolute_error: 

In [24]:
# min(losses_3, key = lambda t: t[2])

(64, (1, 1), 0.22081510424613954)

Dense = 64, Stride = (1,1), Loss = 0.22081510424613954

### Adding a second convolutional layer
### Convolution Size and Dense

In [15]:
conv2d_size = [int(x) for x in [2**y for y in np.arange(1, 9, 1)]]
dense_size = [int(x) for x in [2**y for y in np.arange(1, 9, 1)]]
losses_4 = []

In [16]:
START_LEARNING_RATE = 0.0024787521766663585
NUM_EPOCHS = 10
BATCH_SIZE = 1000

def model_function(conv2d, dense):
    # a layer instance is callable on a tensor, and returns a tensor
    model = tf.keras.Sequential([
        Conv2D(16, kernel_size=(2, 2), strides=(1,1),
                      data_format='channels_last',
                      activation='relu',
                      input_shape=(13, 13, 3)),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(conv2d, kernel_size=(2, 2), strides=(1,1),
                      activation='relu'),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Flatten(),
        Dense(dense, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(START_LEARNING_RATE),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
    
    return model

In [18]:
# for conv in conv2d_size:
#     print("CONV: ", conv)
#     for dense in dense_size:
#         print("DENSE: ", dense)
#         model = model_function(conv, dense)

#         model.fit(X_train_3,y_train, verbose=1, batch_size = BATCH_SIZE,
#                   epochs=NUM_EPOCHS)
#         evaluate = model.evaluate(X_test_3, y_test, batch_size = BATCH_SIZE)
#         losses_4.append((conv, dense, evaluate[0]))

CONV:  2
DENSE:  2
Epoch 1/10
10000/10000 [==============================] - 1s 58us/step - loss: 0.2497 - mean_absolute_error: 0.4995
Epoch 2/10
10000/10000 [==============================] - 0s 19us/step - loss: 0.2492 - mean_absolute_error: 0.4990
Epoch 3/10
10000/10000 [==============================] - 0s 21us/step - loss: 0.2489 - mean_absolute_error: 0.4982
Epoch 4/10
10000/10000 [==============================] - 0s 21us/step - loss: 0.2484 - mean_absolute_error: 0.4976
Epoch 5/10
10000/10000 [==============================] - 0s 19us/step - loss: 0.2479 - mean_absolute_error: 0.4969
Epoch 6/10
10000/10000 [==============================] - 0s 19us/step - loss: 0.2475 - mean_absolute_error: 0.4962
Epoch 7/10
10000/10000 [==============================] - 0s 20us/step - loss: 0.2471 - mean_absolute_error: 0.4956
Epoch 8/10
10000/10000 [==============================] - 0s 19us/step - loss: 0.2467 - mean_absolute_error: 0.4948
Epoch 9/10
10000/10000 [=============================

10000/10000 [==============================] - 0s 21us/step - loss: 0.2462 - mean_absolute_error: 0.4940
Epoch 4/10
10000/10000 [==============================] - 0s 20us/step - loss: 0.2448 - mean_absolute_error: 0.4915
Epoch 5/10
10000/10000 [==============================] - 0s 22us/step - loss: 0.2433 - mean_absolute_error: 0.4885
Epoch 6/10
10000/10000 [==============================] - 0s 22us/step - loss: 0.2420 - mean_absolute_error: 0.4862
Epoch 7/10
10000/10000 [==============================] - 0s 21us/step - loss: 0.2410 - mean_absolute_error: 0.4840
Epoch 8/10
10000/10000 [==============================] - 0s 20us/step - loss: 0.2398 - mean_absolute_error: 0.4814
Epoch 9/10
10000/10000 [==============================] - 0s 21us/step - loss: 0.2387 - mean_absolute_error: 0.4790
Epoch 10/10
10000/10000 [==============================] - 0s 38us/step
DENSE:  128
Epoch 1/10
10000/10000 [==============================] - 1s 97us/step - loss: 0.2487 - mean_absolute_error: 0.4977

10000/10000 [==============================] - 0s 22us/step - loss: 0.2465 - mean_absolute_error: 0.4955
Epoch 6/10
10000/10000 [==============================] - 0s 23us/step - loss: 0.2450 - mean_absolute_error: 0.4930
Epoch 7/10
10000/10000 [==============================] - 0s 22us/step - loss: 0.2426 - mean_absolute_error: 0.4894
Epoch 8/10
10000/10000 [==============================] - 0s 23us/step - loss: 0.2405 - mean_absolute_error: 0.4857
Epoch 9/10
10000/10000 [==============================] - 0s 22us/step - loss: 0.2375 - mean_absolute_error: 0.4807
Epoch 10/10
10000/10000 [==============================] - 1s 50us/step
DENSE:  16
Epoch 1/10
10000/10000 [==============================] - 1s 131us/step - loss: 0.2512 - mean_absolute_error: 0.5003
Epoch 2/10
10000/10000 [==============================] - 0s 24us/step - loss: 0.2488 - mean_absolute_error: 0.4981
Epoch 3/10
10000/10000 [==============================] - 0s 24us/step - loss: 0.2475 - mean_absolute_error: 0.4969

10000/10000 [==============================] - 0s 27us/step - loss: 0.2234 - mean_absolute_error: 0.4535
Epoch 8/10
10000/10000 [==============================] - 0s 26us/step - loss: 0.2220 - mean_absolute_error: 0.4473
Epoch 9/10
10000/10000 [==============================] - 0s 27us/step - loss: 0.2169 - mean_absolute_error: 0.4419
Epoch 10/10
10000/10000 [==============================] - 1s 72us/step
CONV:  64
DENSE:  2
Epoch 1/10
10000/10000 [==============================] - 2s 205us/step - loss: 0.2506 - mean_absolute_error: 0.4996
Epoch 2/10
10000/10000 [==============================] - 1s 58us/step - loss: 0.2499 - mean_absolute_error: 0.4999
Epoch 3/10
10000/10000 [==============================] - 0s 38us/step - loss: 0.2496 - mean_absolute_error: 0.4996
Epoch 4/10
10000/10000 [==============================] - 0s 28us/step - loss: 0.2489 - mean_absolute_error: 0.4988
Epoch 5/10
10000/10000 [==============================] - 0s 30us/step - loss: 0.2479 - mean_absolute_erro

10000/10000 [==============================] - 0s 39us/step - loss: 0.2199 - mean_absolute_error: 0.4453
Epoch 10/10
10000/10000 [==============================] - 1s 112us/step
DENSE:  64
Epoch 1/10
10000/10000 [==============================] - 2s 250us/step - loss: 0.2533 - mean_absolute_error: 0.4991
Epoch 2/10
10000/10000 [==============================] - 0s 40us/step - loss: 0.2475 - mean_absolute_error: 0.4968
Epoch 3/10
10000/10000 [==============================] - 0s 40us/step - loss: 0.2441 - mean_absolute_error: 0.4916
Epoch 4/10
10000/10000 [==============================] - 0s 38us/step - loss: 0.2397 - mean_absolute_error: 0.4843
Epoch 5/10
10000/10000 [==============================] - 0s 39us/step - loss: 0.2347 - mean_absolute_error: 0.4745
Epoch 6/10
10000/10000 [==============================] - 0s 40us/step - loss: 0.2303 - mean_absolute_error: 0.4653
Epoch 7/10
10000/10000 [==============================] - 0s 40us/step - loss: 0.2257 - mean_absolute_error: 0.458

In [19]:
# min(losses_4, key = lambda t: t[2])

(256, 32, 0.22602192908525467)

Conv = 256, dense = 32, loss = 0.22602192908525467

### Kernel Size and Stride for Second Layer

In [17]:
START_LEARNING_RATE = 0.0024787521766663585
NUM_EPOCHS = 10
BATCH_SIZE = 1000

def model_function(kernel, stride):
    # a layer instance is callable on a tensor, and returns a tensor
    model = tf.keras.Sequential([
        Conv2D(16, kernel_size=(2, 2), strides=(1,1),
                      data_format='channels_last',
                      activation='relu',
                      input_shape=(13, 13, 3)),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(256, kernel_size=kernel, strides=stride,
                      activation='relu'),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Flatten(),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(START_LEARNING_RATE),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
    
    return model

In [18]:
strides = [(1,1), (1,2), (1,3), (2,1), (2,2), (2,3), (3,1), (3,2), (3,3)]
kernel_sizes = [(1,2), (1,3), (2,1), (2,2), (2,3), (3,1), (3,2), (3,3)]
losses_5 = []

In [23]:
# for stride in strides:
#     print("STRIDE: ", stride)
#     for kernel in kernel_sizes:
#         print("KERNEL: ", kernel)
#         model = model_function(kernel, stride)

#         model.fit(X_train_3,y_train, verbose=1, batch_size = BATCH_SIZE,
#                   epochs=NUM_EPOCHS)
#         evaluate = model.evaluate(X_test_3, y_test, batch_size = BATCH_SIZE)
#         losses_5.append((stride, kernel, evaluate[0]))

STRIDE:  (1, 1)
KERNEL:  (1, 2)
Epoch 1/10
10000/10000 [==============================] - 3s 343us/step - loss: 0.2529 - mean_absolute_error: 0.4980
Epoch 2/10
10000/10000 [==============================] - 1s 71us/step - loss: 0.2457 - mean_absolute_error: 0.4947
Epoch 3/10
10000/10000 [==============================] - 1s 70us/step - loss: 0.2401 - mean_absolute_error: 0.4861
Epoch 4/10
10000/10000 [==============================] - 1s 72us/step - loss: 0.2342 - mean_absolute_error: 0.4730
Epoch 5/10
10000/10000 [==============================] - 1s 71us/step - loss: 0.2296 - mean_absolute_error: 0.4633
Epoch 6/10
10000/10000 [==============================] - 1s 72us/step - loss: 0.2248 - mean_absolute_error: 0.4564
Epoch 7/10
10000/10000 [==============================] - 1s 74us/step - loss: 0.2209 - mean_absolute_error: 0.4492
Epoch 8/10
10000/10000 [==============================] - 1s 72us/step - loss: 0.2179 - mean_absolute_error: 0.4425
Epoch 9/10
10000/10000 [===============

10000/10000 [==============================] - 1s 61us/step - loss: 0.2486 - mean_absolute_error: 0.4982
Epoch 3/10
10000/10000 [==============================] - 0s 42us/step - loss: 0.2470 - mean_absolute_error: 0.4954
Epoch 4/10
10000/10000 [==============================] - 0s 40us/step - loss: 0.2454 - mean_absolute_error: 0.4927
Epoch 5/10
10000/10000 [==============================] - 0s 43us/step - loss: 0.2418 - mean_absolute_error: 0.4877
Epoch 6/10
10000/10000 [==============================] - 0s 41us/step - loss: 0.2406 - mean_absolute_error: 0.4842
Epoch 7/10
10000/10000 [==============================] - 1s 62us/step - loss: 0.2383 - mean_absolute_error: 0.4795
Epoch 8/10
10000/10000 [==============================] - 1s 69us/step - loss: 0.2353 - mean_absolute_error: 0.4752
Epoch 9/10
10000/10000 [==============================] - 1s 61us/step - loss: 0.2339 - mean_absolute_error: 0.4713
Epoch 10/10
10000/10000 [==============================] - 2s 158us/step
KERNEL:  (

10000/10000 [==============================] - 1s 87us/step - loss: 0.2462 - mean_absolute_error: 0.4948
Epoch 4/10
10000/10000 [==============================] - 1s 88us/step - loss: 0.2439 - mean_absolute_error: 0.4909
Epoch 5/10
10000/10000 [==============================] - 1s 65us/step - loss: 0.2420 - mean_absolute_error: 0.4869
Epoch 6/10
10000/10000 [==============================] - 1s 76us/step - loss: 0.2389 - mean_absolute_error: 0.4824
Epoch 7/10
10000/10000 [==============================] - 1s 72us/step - loss: 0.2353 - mean_absolute_error: 0.4727
Epoch 8/10
10000/10000 [==============================] - 1s 65us/step - loss: 0.2333 - mean_absolute_error: 0.4680
Epoch 9/10
10000/10000 [==============================] - 1s 52us/step - loss: 0.2324 - mean_absolute_error: 0.4671
Epoch 10/10
10000/10000 [==============================] - 2s 200us/step
KERNEL:  (2, 2)
Epoch 1/10
10000/10000 [==============================] - 5s 460us/step - loss: 0.2517 - mean_absolute_error: 

Epoch 5/10
10000/10000 [==============================] - 0s 41us/step - loss: 0.2348 - mean_absolute_error: 0.4706
Epoch 6/10
10000/10000 [==============================] - 1s 60us/step - loss: 0.2327 - mean_absolute_error: 0.4674
Epoch 7/10
10000/10000 [==============================] - 1s 65us/step - loss: 0.2304 - mean_absolute_error: 0.4645
Epoch 8/10
10000/10000 [==============================] - 1s 60us/step - loss: 0.2260 - mean_absolute_error: 0.4581
Epoch 9/10
10000/10000 [==============================] - 0s 42us/step - loss: 0.2220 - mean_absolute_error: 0.4517
Epoch 10/10
10000/10000 [==============================] - 2s 237us/step
STRIDE:  (2, 3)
KERNEL:  (1, 2)
Epoch 1/10
10000/10000 [==============================] - 5s 517us/step - loss: 0.2494 - mean_absolute_error: 0.4978
Epoch 2/10
10000/10000 [==============================] - 0s 38us/step - loss: 0.2468 - mean_absolute_error: 0.4948
Epoch 3/10
10000/10000 [==============================] - 0s 40us/step - loss: 0.2

10000/10000 [==============================] - 1s 81us/step - loss: 0.2236 - mean_absolute_error: 0.4523
Epoch 7/10
10000/10000 [==============================] - 1s 69us/step - loss: 0.2193 - mean_absolute_error: 0.4464
Epoch 8/10
10000/10000 [==============================] - 1s 70us/step - loss: 0.2151 - mean_absolute_error: 0.4388
Epoch 9/10
10000/10000 [==============================] - 1s 61us/step - loss: 0.2111 - mean_absolute_error: 0.4320
Epoch 10/10
10000/10000 [==============================] - 3s 271us/step
KERNEL:  (3, 1)
Epoch 1/10
10000/10000 [==============================] - 6s 633us/step - loss: 0.2505 - mean_absolute_error: 0.4978
Epoch 2/10
10000/10000 [==============================] - 1s 73us/step - loss: 0.2426 - mean_absolute_error: 0.4896
Epoch 3/10
10000/10000 [==============================] - 1s 57us/step - loss: 0.2367 - mean_absolute_error: 0.4781
Epoch 4/10
10000/10000 [==============================] - 0s 46us/step - loss: 0.2322 - mean_absolute_error: 

10000/10000 [==============================] - 1s 60us/step - loss: 0.2347 - mean_absolute_error: 0.4734
Epoch 8/10
10000/10000 [==============================] - 1s 63us/step - loss: 0.2334 - mean_absolute_error: 0.4709
Epoch 9/10
10000/10000 [==============================] - 0s 40us/step - loss: 0.2314 - mean_absolute_error: 0.4672
Epoch 10/10
10000/10000 [==============================] - 3s 311us/step
KERNEL:  (2, 1)
Epoch 1/10
10000/10000 [==============================] - 7s 686us/step - loss: 0.2508 - mean_absolute_error: 0.4997
Epoch 2/10
10000/10000 [==============================] - 0s 38us/step - loss: 0.2492 - mean_absolute_error: 0.4988
Epoch 3/10
10000/10000 [==============================] - 0s 37us/step - loss: 0.2477 - mean_absolute_error: 0.4970
Epoch 4/10
10000/10000 [==============================] - 0s 38us/step - loss: 0.2462 - mean_absolute_error: 0.4945
Epoch 5/10
10000/10000 [==============================] - 0s 39us/step - loss: 0.2443 - mean_absolute_error: 

In [29]:
# min(losses_5, key = lambda t: t[2])

((1, 1), (3, 3), 0.22120582014322282)

stride = (1,1), kernel = (3,3), loss = 0.22120582014322282

### Kernel Size and Stride for the First layer

In [19]:
START_LEARNING_RATE = 0.0024787521766663585
NUM_EPOCHS = 10
BATCH_SIZE = 1000

def model_function(kernel, stride):
    # a layer instance is callable on a tensor, and returns a tensor
    model = tf.keras.Sequential([
        Conv2D(16, kernel_size=kernel, strides=stride,
                      data_format='channels_last',
                      activation='relu',
                      input_shape=(13, 13, 3)),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(256, kernel_size=(3,3), strides=(1,1),
                      activation='relu'),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Flatten(),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(START_LEARNING_RATE),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
    
    return model

In [37]:
strides = [(1,1)]
kernel_sizes = [(2,2), (3,3), (4,4)]
losses_6 = []

In [38]:
# for stride in strides:
#     print("STRIDE: ", stride)
#     for kernel in kernel_sizes:
#         print("KERNEL: ", kernel)
#         if(stride[0] == 2 and stride[1] == 2 and kernel[0]==2 and kernel[1] == 2):
#             print("no")
#         else:
#             model = model_function(kernel, stride)

#             model.fit(X_train_3,y_train, verbose=1, batch_size = BATCH_SIZE,
#                       epochs=NUM_EPOCHS)
#             evaluate = model.evaluate(X_test_3, y_test, batch_size = BATCH_SIZE)
#             losses_6.append((stride, kernel, evaluate[0]))

STRIDE:  (1, 1)
KERNEL:  (2, 2)
Epoch 1/10
10000/10000 [==============================] - 2s 164us/step - loss: 0.2531 - mean_absolute_error: 0.4982
Epoch 2/10
10000/10000 [==============================] - 1s 70us/step - loss: 0.2455 - mean_absolute_error: 0.4938
Epoch 3/10
10000/10000 [==============================] - 1s 57us/step - loss: 0.2413 - mean_absolute_error: 0.4859
Epoch 4/10
10000/10000 [==============================] - 1s 61us/step - loss: 0.2355 - mean_absolute_error: 0.4775
Epoch 5/10
10000/10000 [==============================] - 1s 64us/step - loss: 0.2286 - mean_absolute_error: 0.4658
Epoch 6/10
10000/10000 [==============================] - 1s 87us/step - loss: 0.2242 - mean_absolute_error: 0.4547
Epoch 7/10
10000/10000 [==============================] - 1s 88us/step - loss: 0.2181 - mean_absolute_error: 0.4445
Epoch 8/10
10000/10000 [==============================] - 1s 102us/step - loss: 0.2158 - mean_absolute_error: 0.4382
Epoch 9/10
10000/10000 [==============

In [39]:
# min(losses_6, key = lambda t: t[2])

((1, 1), (2, 2), 0.22206038087606431)

((1, 1), (2, 2), 0.0.22206038087606431)

### Try adding one more layer (3rd)
### Conv, conv2d

In [40]:
conv2d_size = [int(x) for x in [2**y for y in np.arange(1, 8, 1)]]
dense_size = [int(x) for x in [2**y for y in np.arange(1, 8, 1)]]
# kernels = [(2,2), (3,3)]
losses_7 = []

In [48]:
START_LEARNING_RATE = 0.0024787521766663585
NUM_EPOCHS = 10
BATCH_SIZE = 1000

def model_function(conv, dense):
    # a layer instance is callable on a tensor, and returns a tensor
    model = tf.keras.Sequential([
        Conv2D(16, kernel_size=(2,2), strides=(1,1),
                      data_format='channels_last',
                      activation='relu',
                      input_shape=(13, 13, 3)),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(256, kernel_size=(3,3), strides=(1,1), activation='relu'),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(conv, kernel_size=(2,2), strides=(1,1), activation='relu'),
        MaxPool2D(pool_size=(1, 1), strides=(1, 1)),
        Flatten(),
        Dense(dense, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(START_LEARNING_RATE),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
    
    return model

In [49]:
# for conv in conv2d_size:
#     print("CONV: ", conv)
#     for dense in dense_size:
#         print("DENSE: ", dense)
#         model = model_function(conv, dense)

#         model.fit(X_train_3,y_train, verbose=1, batch_size = BATCH_SIZE,
#                       epochs=NUM_EPOCHS)
#         evaluate = model.evaluate(X_test_3, y_test, batch_size = BATCH_SIZE)
#         losses_7.append((conv, dense, evaluate[0]))

CONV:  2
DENSE:  2
Epoch 1/10
10000/10000 [==============================] - 2s 159us/step - loss: 0.2500 - mean_absolute_error: 0.5000
Epoch 2/10
10000/10000 [==============================] - 1s 109us/step - loss: 0.2500 - mean_absolute_error: 0.5000
Epoch 3/10
10000/10000 [==============================] - 1s 111us/step - loss: 0.2500 - mean_absolute_error: 0.5000
Epoch 4/10
10000/10000 [==============================] - 1s 83us/step - loss: 0.2500 - mean_absolute_error: 0.5000
Epoch 5/10
10000/10000 [==============================] - 1s 100us/step - loss: 0.2500 - mean_absolute_error: 0.5000
Epoch 6/10
10000/10000 [==============================] - 1s 112us/step - loss: 0.2500 - mean_absolute_error: 0.5000
Epoch 7/10
10000/10000 [==============================] - 1s 98us/step - loss: 0.2500 - mean_absolute_error: 0.5000
Epoch 8/10
10000/10000 [==============================] - 1s 78us/step - loss: 0.2500 - mean_absolute_error: 0.5000
Epoch 9/10
10000/10000 [========================

Epoch 3/10
10000/10000 [==============================] - 1s 72us/step - loss: 0.2334 - mean_absolute_error: 0.4740
Epoch 4/10
10000/10000 [==============================] - 1s 79us/step - loss: 0.2239 - mean_absolute_error: 0.4576
Epoch 5/10
10000/10000 [==============================] - 1s 70us/step - loss: 0.2142 - mean_absolute_error: 0.4372
Epoch 6/10
10000/10000 [==============================] - 1s 93us/step - loss: 0.2097 - mean_absolute_error: 0.4244
Epoch 7/10
10000/10000 [==============================] - 1s 105us/step - loss: 0.2031 - mean_absolute_error: 0.4132
Epoch 8/10
10000/10000 [==============================] - 1s 86us/step - loss: 0.1994 - mean_absolute_error: 0.4060
Epoch 9/10
10000/10000 [==============================] - 1s 69us/step - loss: 0.1899 - mean_absolute_error: 0.3949
Epoch 10/10
10000/10000 [==============================] - 1s 87us/step
CONV:  8
DENSE:  2
Epoch 1/10
10000/10000 [==============================] - 2s 234us/step - loss: 0.2466 - mean_ab

10000/10000 [==============================] - 1s 82us/step - loss: 0.2176 - mean_absolute_error: 0.4456
Epoch 6/10
10000/10000 [==============================] - 1s 102us/step - loss: 0.2101 - mean_absolute_error: 0.4321
Epoch 7/10
10000/10000 [==============================] - 1s 104us/step - loss: 0.2037 - mean_absolute_error: 0.4154
Epoch 8/10
10000/10000 [==============================] - 1s 106us/step - loss: 0.1999 - mean_absolute_error: 0.4086
Epoch 9/10
10000/10000 [==============================] - 1s 80us/step - loss: 0.1951 - mean_absolute_error: 0.4008
Epoch 10/10
10000/10000 [==============================] - 1s 131us/step
DENSE:  128
Epoch 1/10
10000/10000 [==============================] - 3s 293us/step - loss: 0.2496 - mean_absolute_error: 0.4989
Epoch 2/10
10000/10000 [==============================] - 1s 71us/step - loss: 0.2445 - mean_absolute_error: 0.4907
Epoch 3/10
10000/10000 [==============================] - 1s 99us/step - loss: 0.2362 - mean_absolute_error: 0

Epoch 1/10
10000/10000 [==============================] - 3s 326us/step - loss: 0.2471 - mean_absolute_error: 0.4931
Epoch 2/10
10000/10000 [==============================] - 1s 72us/step - loss: 0.2369 - mean_absolute_error: 0.4797
Epoch 3/10
10000/10000 [==============================] - 1s 72us/step - loss: 0.2266 - mean_absolute_error: 0.4615
Epoch 4/10
10000/10000 [==============================] - 1s 70us/step - loss: 0.2184 - mean_absolute_error: 0.4422
Epoch 5/10
10000/10000 [==============================] - 1s 73us/step - loss: 0.2153 - mean_absolute_error: 0.4316
Epoch 6/10
10000/10000 [==============================] - 1s 79us/step - loss: 0.2147 - mean_absolute_error: 0.4321
Epoch 7/10
10000/10000 [==============================] - 1s 69us/step - loss: 0.2055 - mean_absolute_error: 0.4224
Epoch 8/10
10000/10000 [==============================] - 1s 108us/step - loss: 0.1985 - mean_absolute_error: 0.4072
Epoch 9/10
10000/10000 [==============================] - 1s 125us/ste

10000/10000 [==============================] - 1s 147us/step - loss: 0.2459 - mean_absolute_error: 0.4948
Epoch 3/10
10000/10000 [==============================] - 1s 143us/step - loss: 0.2388 - mean_absolute_error: 0.4814
Epoch 4/10
10000/10000 [==============================] - 1s 149us/step - loss: 0.2344 - mean_absolute_error: 0.4687
Epoch 5/10
10000/10000 [==============================] - 2s 162us/step - loss: 0.2262 - mean_absolute_error: 0.4626
Epoch 6/10
10000/10000 [==============================] - 1s 122us/step - loss: 0.2187 - mean_absolute_error: 0.4452
Epoch 7/10
10000/10000 [==============================] - 1s 88us/step - loss: 0.2109 - mean_absolute_error: 0.4303
Epoch 8/10
10000/10000 [==============================] - 1s 91us/step - loss: 0.2048 - mean_absolute_error: 0.4179
Epoch 9/10
10000/10000 [==============================] - 1s 89us/step - loss: 0.1967 - mean_absolute_error: 0.4059
Epoch 10/10
10000/10000 [==============================] - 2s 180us/step
DENSE

In [50]:
# min(losses_7, key = lambda t: t[2])

(8, 2, 0.21907085925340652)

(8, 2, 0.21907085925340652)

### Trying a couple options with 200,000

In [20]:
SAMPLE_SIZE = 200000
with open('data/board_states.npy', 'rb') as boards_file, \
    open('data/winners.npy', 'rb') as winners_file:
    boards = np.load(boards_file)
    shape = boards.shape
    boards = np.array([np.subtract(x[:,:,0], x[:,:,1]) for x in boards]).reshape((shape[0], shape[1], shape[2], 1))
    
    #3 channels
    boards_3 = np.tile(boards,3)
    
#     winners = np.load(winners_file)
    print(boards_3[:SAMPLE_SIZE, :, :, :].shape, winners[:SAMPLE_SIZE, :].shape)
    
    X_train_3 = boards_3[nums[:SAMPLE_SIZE],:,:,:]

    y_train = winners[nums[:SAMPLE_SIZE],:]
    
    X_test_3 = boards_3[nums[SAMPLE_SIZE:(2*SAMPLE_SIZE)],:,:,:]
    
    y_test = winners[nums[SAMPLE_SIZE:(2*SAMPLE_SIZE)],:]

(200000, 13, 13, 3) (200000, 1)


In [21]:
START_LEARNING_RATE = 0.0024787521766663585
NUM_EPOCHS = 10
BATCH_SIZE = 1000

def model_function():
    # a layer instance is callable on a tensor, and returns a tensor
    model = tf.keras.Sequential([
        Conv2D(16, kernel_size=(2,2), strides=(1,1),
                      data_format='channels_last',
                      activation='relu',
                      input_shape=(13, 13, 3)),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(256, kernel_size=(3,3), strides=(1,1), activation='relu'),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(8, kernel_size=(2,2), strides=(1,1), activation='relu'),
        MaxPool2D(pool_size=(1, 1), strides=(1, 1)),
        Flatten(),
        Dense(2, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(START_LEARNING_RATE),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
    
    return model

In [22]:
model = model_function()
model.fit(X_train_3,y_train, verbose=1, batch_size = BATCH_SIZE,
              epochs=NUM_EPOCHS)
evaluate = model.evaluate(X_test_3, y_test, batch_size = BATCH_SIZE)
# predict = model.predict(X_test)
# r2_score(y_test, y_pred)

evaluate[0]

Epoch 1/10
200000/200000 [==============================] - 14s 72us/step - loss: 0.2319 - mean_absolute_error: 0.4689
Epoch 2/10
200000/200000 [==============================] - 18s 92us/step - loss: 0.2137 - mean_absolute_error: 0.4367
Epoch 3/10
200000/200000 [==============================] - 15s 76us/step - loss: 0.2020 - mean_absolute_error: 0.4145
Epoch 4/10
200000/200000 [==============================] - 14s 69us/step - loss: 0.1915 - mean_absolute_error: 0.3948
Epoch 5/10
200000/200000 [==============================] - 14s 68us/step - loss: 0.1826 - mean_absolute_error: 0.3763
Epoch 6/10
200000/200000 [==============================] - 18s 92us/step - loss: 0.1753 - mean_absolute_error: 0.3619
Epoch 7/10
200000/200000 [==============================] - 16s 79us/step - loss: 0.1676 - mean_absolute_error: 0.3470
Epoch 8/10
200000/200000 [==============================] - 14s 70us/step - loss: 0.1613 - mean_absolute_error: 0.3336
Epoch 9/10
200000/200000 [======================

0.17362344749271869

In [23]:
START_LEARNING_RATE = 0.0024787521766663585
NUM_EPOCHS = 10
BATCH_SIZE = 1000

def model_function():
    # a layer instance is callable on a tensor, and returns a tensor
    model = tf.keras.Sequential([
        Conv2D(16, kernel_size=(2,2), strides=(1,1),
                      data_format='channels_last',
                      activation='relu',
                      input_shape=(13, 13, 3)),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(256, kernel_size=(3,3), strides=(1,1),
                      activation='relu'),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Flatten(),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(START_LEARNING_RATE),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
    
    return model

In [24]:
model = model_function()
model.fit(X_train_3,y_train, verbose=1, batch_size = BATCH_SIZE,
              epochs=NUM_EPOCHS)
evaluate = model.evaluate(X_test_3, y_test, batch_size = BATCH_SIZE)
# predict = model.predict(X_test)
# r2_score(y_test, y_pred)

evaluate[0]

Epoch 1/10
200000/200000 [==============================] - 13s 67us/step - loss: 0.2242 - mean_absolute_error: 0.4503
Epoch 2/10
200000/200000 [==============================] - 12s 61us/step - loss: 0.2046 - mean_absolute_error: 0.4120
Epoch 3/10
200000/200000 [==============================] - 17s 86us/step - loss: 0.1954 - mean_absolute_error: 0.3936
Epoch 4/10
200000/200000 [==============================] - 15s 76us/step - loss: 0.1856 - mean_absolute_error: 0.3750
Epoch 5/10
200000/200000 [==============================] - 13s 64us/step - loss: 0.1768 - mean_absolute_error: 0.3574
Epoch 6/10
200000/200000 [==============================] - 12s 60us/step - loss: 0.1693 - mean_absolute_error: 0.3417
Epoch 7/10
200000/200000 [==============================] - 16s 82us/step - loss: 0.1627 - mean_absolute_error: 0.3280
Epoch 8/10
200000/200000 [==============================] - 16s 81us/step - loss: 0.1575 - mean_absolute_error: 0.3160
Epoch 9/10
200000/200000 [======================

0.16840128466486931

In [25]:
START_LEARNING_RATE = 0.0024787521766663585
NUM_EPOCHS = 10
BATCH_SIZE = 1000

def model_function():
    # a layer instance is callable on a tensor, and returns a tensor
    model = tf.keras.Sequential([
        Conv2D(16, kernel_size=(2, 2), strides=(1,1),
                      data_format='channels_last',
                      activation='relu',
                      input_shape=(13, 13, 3)),
        MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer=tf.train.AdamOptimizer(START_LEARNING_RATE),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
    
    return model

In [26]:
model = model_function()
model.fit(X_train_3,y_train, verbose=1, batch_size = BATCH_SIZE,
              epochs=NUM_EPOCHS)
evaluate = model.evaluate(X_test_3, y_test, batch_size = BATCH_SIZE)
# predict = model.predict(X_test)
# r2_score(y_test, y_pred)

evaluate[0]

Epoch 1/10
200000/200000 [==============================] - 6s 31us/step - loss: 0.2216 - mean_absolute_error: 0.4451
Epoch 2/10
200000/200000 [==============================] - 6s 32us/step - loss: 0.2057 - mean_absolute_error: 0.4145
Epoch 3/10
200000/200000 [==============================] - 7s 34us/step - loss: 0.1963 - mean_absolute_error: 0.3954
Epoch 4/10
200000/200000 [==============================] - 7s 35us/step - loss: 0.1879 - mean_absolute_error: 0.3794
Epoch 5/10
200000/200000 [==============================] - 5s 25us/step - loss: 0.1799 - mean_absolute_error: 0.3635
Epoch 6/10
200000/200000 [==============================] - 5s 26us/step - loss: 0.1733 - mean_absolute_error: 0.3500
Epoch 7/10
200000/200000 [==============================] - 4s 18us/step - loss: 0.1666 - mean_absolute_error: 0.3368
Epoch 8/10
200000/200000 [==============================] - 4s 19us/step - loss: 0.1617 - mean_absolute_error: 0.3264
Epoch 9/10
200000/200000 [==============================

0.1742646062374115

In [37]:
model.save("./natalie_model")